# Análise Exploratória dos Dados (EDA) 

* **EDA** - Exploratory Data Analysis.

## Exemplo

* Uma imobiliária deseja construir um modelo que estime o preço do imóvel (**ALVO**/ **TARGET**) com base em variáveis relacionadas a residência (**COVARIÁVEIS**/ **FEATURES**).
  
## Objetivo

* Identificar o tipo e a distribuição das variáveis envolvidas, possibilidade de criação de novas variáveis com base nas existentes, resumir informações, detectar padrões com os dados, selecionar variáveis para o desenvolvimento de um modelo, gerar *insights* para a tomada de decisão (definir estratégias de negócio) e mensurar a distribuição conjunta (bivariada) - quantificar a relação entre o ALVO e as COVARIÁVEIS.

**Imports requeridos para a execução do estudo**

In [ ]:
#Bibliotecas básicas
import pandas as pd     #Manipulação dos dados
import numpy as np      #Operações multidimensionais e matemáticas
import matplotlib.pyplot as plt    #Gráficos
import matplotlib.ticker as ticker #Remover a notação científica do gráfico
import seaborn as sns              #Gráficos
##Eliminar os warnings
import warnings
warnings.filterwarnings("ignore")
##Ver todas as colunas do data frame
pd.set_option('display.max_columns', None)
#Definir o formato de exibição tipo float para evitar notação científica
pd.options.display.float_format = '{:.2f}'.format

**Versão do Python instalada**

In [ ]:
#Identificar a versão do Python
import sys
print(f"Versão do Python: {sys.version}") #Versão do Python: 3.11.8

## 1 - Visão geral os dados

In [ ]:
#Importar a base de dados
df =  pd.read_csv('base_dados.csv', sep = ';')
#Visualizar
df.head(3) #Visualizar as três primeiras linhas da tabela

* NaN - **not as number** (é um valor nulo/ *missing*).

In [ ]:
#Características básicas do data frame - Parte 1
df.shape #545 observações e 14 variáveis 

In [ ]:
#Características básicas do data frame - Parte 2
df.info()#Nome da variável, contagem de observações não nulas e o tipo das variáveis

# 2 - Desenvolvimento da EDA

**Percentual de valores nulos para cada variável da base de dados**

In [ ]:
#Calcular o percentual de valores nulos (vetor)
percent_missing = df.isnull().mean() * 100
####################
#Construir uma tabela
df_perc_missing = percent_missing.reset_index()  #Reinicializa o índice em uma coluna
df_perc_missing = df_perc_missing.rename(columns={'index': 'VARIAVEL', 0: 'PERCENTUAL_MISSING'}) #Renomear o nome das colunas
#Visualizar
df_perc_missing

* **Nota**: Se o percentual de *missing* de uma variável for muito elevado (superior a 70% - por exemplo), pode ser mais interessante removê-la da análise.
* **Nota**: Como lidar (e qual o impacto) com valores ausentes de uma variável na modelagem?

## 2.1 - Análise descritiva da variável PRECO

In [ ]:
#Sumário estatístico
df['PRECO'].describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9])

In [ ]:
#Coeficiente de variação - variabilidade em torno da média
round(100 * df['PRECO'].std()/df['PRECO'].mean(),2) #39.24 #Variabilidade em nível mediano

### 2.2.1 - Representação gráfica

In [ ]:
#Boxplot
boxplot = df.boxplot(column='PRECO', grid=False,  patch_artist=True, boxprops=dict(facecolor='lightblue'), medianprops=dict(color='red'))
boxplot.set_title('Distribuição dos preços dos imóveis', fontweight='bold',  loc='left')

#Eixo Y
plt.ylabel('Valores em US$')

#Eixo X
plt.xticks([])
plt.xlabel('Preço do Imóvel')


#Personalizar o eixo y para evitar a notação científica
boxplot.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.0f}'.format(x)))

In [ ]:
#Identificar as observações que são outliers
q1 = np.percentile(df.PRECO, 25) #Primeiro Quartil
q3 = np.percentile(df.PRECO, 75) #Terceiro Quartil

#Calcular a amplitude interquartil (IQR)
iqr = q3 - q1

#Definir os limites para identificação de outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

#Identificar outliers
outliers = [x for x in df.PRECO if x < lower_bound or x > upper_bound]

#Visualização
print("Outliers:", outliers)

In [ ]:
#Total de observações que são outliers
len(outliers) #15

In [ ]:
#A partir de que valor a observação é um outlier
upper_bound = q3 + 1.5 * iqr
upper_bound #9.205.000

**Nota**: Como fica a previsão do PRECO para as observações que são outliers?

## 2.3 - Análise das variáveis explicativas quantitativas 

In [ ]:
#Extraíndo as variáveis que são do tipo FLOAT e INT
tipos_desejados = ['int64', 'float64']
colunas_desejadas = [coluna for coluna in df.select_dtypes(include=tipos_desejados) if coluna != 'PRECO']

In [ ]:
#EDA das variáveis numéricas
for coluna in colunas_desejadas:
    print(f"--- {coluna} ---")
    print(f"Média: {df[coluna].mean()}")
    print(f"Mediana: {df[coluna].median()}")
    print(f"Máximo: {df[coluna].max()}")
    print(f"Mínimo: {df[coluna].min()}")
    print(f"Desvio Padrão: {df[coluna].std()}")
    print(f"Coeficiente de variação: {df[coluna].std()/df[coluna].mean() * 100}")
    print()

**Nota**: Saber **identificar a dispersão e o seu range de valores (variabilidade e cardinalidade)**. 
* Exemplo: A variável *VAGAS_ESTACIONAMENTO*, observa-se que muitas casas tem poucas ou nenhuma vaga de estacionamento para carros e poucas residências tem muitas - **alta dispersão**.

## 2.4 - Análise descritiva das variáveis categóricas 

In [ ]:
#As classes de cada variável categórica e sua frequência relativa
for coluna in df.select_dtypes(include = "object"):
    print (f"{coluna}:\n{df[coluna].value_counts(True)}")

**Nota**: Saber identificar a **frequência relativa das classes da variável categórica**, no sentido de avaliar **pontos de concentração**.
* Exemplo: A variável FLAG_AGUA_MORNA concentra que 95% das casas não tem água morna.

**Nota**: As seções 2.3 e 2.4 representaram análises univariadas. 

**Ideia**: Com as estatísticas obtidas das variáveis independentes seria possível criar uma ***PERSONA*, ou seja, o perfil médio das casas**. Poderia também pensar numa análise de agrupamento ou aplicar regras de negócio no contexto de segmentação.

## 2.5 - Análise bivariada (distribuição conjunta)

* Quantificar o efeito das covariáveis na distribuição da variável resposta. 

**ALVO vs variáveis categóricas**

In [ ]:
#Lista de colunas categóricas
colunas_categoricas = df.select_dtypes(include='object').columns

In [ ]:
#Iterando sobre as colunas categóricas
for coluna in colunas_categoricas:
    print(f"Análise bivariada {coluna}:")
    summary = df.groupby([coluna])['PRECO'].agg(['mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
    summary.columns = ['Média', 'Mediana', 'Desvio Padrão', 'Percentil 25', 'Percentil 75']
    print(summary)
    print('\n')

**Nota**: A distribuição do *PRECO* é muito sensível com a variável *FLAG_CENTRO*.

**ALVO vs variáveis numéricas**

1. **Correlação**
    * Correlação das variáveis explicativas com o ALVO e entre elas.

**Mapa de calor**

In [ ]:
#Heatmap
#Data frame com as colunas numéricas
df_numerical = df.select_dtypes(include=['int', 'float'])
#Correlação
plt.figure(figsize = (7,7))
sns.heatmap(df_numerical.corr("spearman"), annot = True, cmap = "YlGnBu")
plt.title("Mapa de Correlação das Variáveis Numéricas\n", fontsize = 15)
plt.show()

**Tabela de correlação**

In [ ]:
#Tabela de correlação
tabela_correlacao_spearman = df_numerical.corr(method='spearman') #Não preciso assumir uma relação linear entre as variáveis e não tem hipótese sobre a distribuição normal
tabela_correlacao_spearman #A variável AREA tem a maior correlação com preço.

2. **Scatter Plot** (Gráfico de Dispersão)

In [ ]:
#Scatter plot
plt.scatter(df.AREA, df.PRECO)

#Adicionar rótulos aos eixos
plt.xlabel('Área do imóvel')
plt.ylabel('Preço do imóvel')

#Adicionar título ao gráfico
plt.suptitle('Reção entre Área vs Preço', fontweight='bold', x=0.1)

#Remover linhas de grade
plt.grid(False)

#Remover notação científica do eixo y
plt.ticklabel_format(style='plain', axis='y')

#Exibir o gráfico
plt.show()

#### Análise adicional (feature engineering)

* Criar uma variável a partir de uma existente. Posteriormente, compará-la com o ALVO.
* Vou criar a variável chamada **DECIL_AREA**.

In [ ]:
#Variável que vai calcular os decis da variável AREA
decis_area = pd.qcut(df['AREA'], q=10)
#Adicionar a variável DECIL_AREA
df['DECIL_AREA'] = decis_area

In [ ]:
#Calcular o preço médio do imóvel para cada faixa de decil da área
preco_medio_por_decil = df.groupby('DECIL_AREA')['PRECO'].mean()
preco_medio_por_decil #Transformou uma variável quantitativa numa variável qualitativa

In [ ]:
#Visualização final do data frame
df.head(3) #Temos uma nova coluna